In [1]:
import torch
import os

In [2]:
#only in google colab
!git clone https://github.com/Giordano-Cicchetti/MaskTune_NN/

Cloning into 'MaskTune_NN'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 106 (delta 45), reused 67 (delta 28), pack-reused 0
Receiving objects: 100% (106/106), 4.37 MiB | 12.11 MiB/s, done.
Resolving deltas: 100% (45/45), done.


In [2]:
#only on google colab
!cp MaskTune_NN/MNIST/* /content -r
!rm MaskTune_NN/ -r

cp: cannot stat 'MaskTune_NN/MNIST/*': No such file or directory
rm: cannot remove 'MaskTune_NN/': No such file or directory


In [3]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
!pip install grad-cam

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from train import *
train= TrainBaseERM(device)




Start loading data of train dataset in main memory




Start loading data of test dataset in main memory




Start loading data of test dataset in main memory




In [ ]:
train.test_dataset_original.targets[0:1000]

In [ ]:
train.train_erm(epochs=10)

In [ ]:
train.train_erm(resume=True,best_resume_checkpoint_path="best_erm_model.pt",last_resume_checkpoint_path="last_erm_model.pt")

In [8]:
train.test(train.test_loader_original,"best_erm_model.pt")

----------testing the model----------


test epoch 12: 100%|██████████| 79/79 [00:08<00:00,  9.00it/s, loss=2.72, accuracy=0.594]

----------Test accuracy =0.5941455696202531----------


In [9]:
train.test(train.test_loader_biased,"best_erm_model.pt")

----------testing the model----------


test epoch 12: 100%|██████████| 79/79 [00:02<00:00, 37.29it/s, loss=4.25, accuracy=0.31]

----------Test accuracy =0.30963212025316456----------


In [ ]:
!rm data -r

In [28]:
def finetune(best_resume_checkpoint_path: str=None, last_resume_checkpoint_path: str=None) -> None:
  checkpoint = torch.load("best_erm_model.pt")

  if(best_resume_checkpoint_path!=None and last_resume_checkpoint_path!=None):
    train.mask_data(train.train_loader,best_resume_checkpoint_path)
    train.model.load_state_dict(checkpoint['model_state_dict'])
    resume_epoch = checkpoint['epoch'] + 1
    train.optimizer = optim.SGD(
                train.model.parameters(),
                lr=0.01,
                momentum=0.9,
                weight_decay=1e-4
            )
    train.lr_scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    
    train.run_an_epoch(data_loader=train.masked_loader,epoch=resume_epoch,mode="train",device=train.device)
    val_accuracy = train.run_an_epoch(
                data_loader=train.val_loader, epoch=resume_epoch, mode="validation",device=train.device
            )
    torch.save({
                'epoch': resume_epoch,
                'model_state_dict': train.model.state_dict(),
                'optimizer_state_dict': train.optimizer.state_dict(),
                'scheduler_state_dict': train.lr_scheduler.state_dict(),
                'accuracy' : val_accuracy,
                }, "masked_erm_model.pt")
    

finetune(best_resume_checkpoint_path="best_erm_model.pt",last_resume_checkpoint_path="last_erm_model.pt")


validation epoch 13: 100%|██████████| 94/94 [00:01<00:00, 47.86it/s, loss=0.72, accuracy=0.454]


In [7]:
train.test(train.test_loader_original,"masked_erm_model.pt")

----------testing the model----------


test epoch masking: 100%|██████████| 79/79 [00:01<00:00, 49.09it/s, loss=0.741, accuracy=0.484]

----------Test accuracy =0.48388053797468356----------


In [8]:
train.test(train.test_loader_biased,"masked_erm_model.pt")

----------testing the model----------


test epoch masking: 100%|██████████| 79/79 [00:02<00:00, 37.80it/s, loss=0.814, accuracy=0.405]

----------Test accuracy =0.40466772151898733----------
